In [1]:
import numpy as np
from torch.utils.data import DataLoader
from src.model.vae.conv_vae import ConvVAE
import torch
from src.utils.calculate_loss import calculate_test_loss, calculate_loss
from src.utils.load_data import load_data, normalize_data_tensor, recover_data_tensor
device = torch.device('cuda')
vae = ConvVAE(4).to(device)
vae.load_state_dict(torch.load('../../saved_models/VAN/VAE-VAN.torch'))

train,_,test = load_data('VAN')
# mean = torch.tensor(train.mean(axis=(0,2,3))).cuda().float()
max = torch.tensor(np.array([train[:,i].max() for i in range(4)])).cuda().float()

mean = train.mean(axis=(0,2,3))
print(mean)


normalizer=mean
device = torch.device('cuda')

dataloader_train = DataLoader(train, 32, shuffle=True, drop_last=True)
dataloader_test = DataLoader(test, 32, shuffle=False, drop_last=True)


[2.48085591 1.03734006 0.92262824 0.74378674]


In [2]:
train.mean()


1.2961527377521613

calculate train test metrics

In [3]:
metric = ['RMSE', 'MSE', 'MAE']

In [4]:
vae.training=False
vae.eval()

metric = ['RMSE', 'MSE', 'MAE']
loss = {m:[] for m in metric}
for idx, data in enumerate(dataloader_train):
    data = data.cuda().float()
    data_recon,_,_ = vae(normalize_data_tensor(data,normalizer))
    for m in metric:
        l = torch.mean(calculate_loss(recover_data_tensor(data_recon, normalizer),
                                      data, m,
                                      reduction='none'),
                       dim=(0,2,3)).cpu().detach().numpy()
        loss[m].append(l)

for m in metric:
    loss[m] = np.stack(loss[m], axis=0)


In [5]:
loss_means = {m:loss[m].mean(axis=0) for m in metric}
loss_means

{'RMSE': array([1.2402833 , 0.693108  , 0.69167763, 0.3438164 ], dtype=float32),
 'MSE': array([3.8418727, 1.0793787, 1.0022407, 0.4509989], dtype=float32),
 'MAE': array([1.2402833 , 0.693108  , 0.69167763, 0.3438164 ], dtype=float32)}

In [6]:
loss_vars = {m:loss[m].var(axis=0) for m in metric}
loss_vars


{'RMSE': array([0.00163881, 0.00046616, 0.00063117, 0.00012514], dtype=float32),
 'MSE': array([0.12656954, 0.10189484, 0.0066062 , 0.00087829], dtype=float32),
 'MAE': array([0.00163881, 0.00046616, 0.00063117, 0.00012514], dtype=float32)}

In [7]:
loss_test = {m:[] for m in metric}
for idx, data in enumerate(dataloader_test):
    data = data.cuda().float()
    data_recon,_,_ = vae(normalize_data_tensor(data,normalizer))
    for m in metric:
        l = torch.mean(calculate_loss(recover_data_tensor(data_recon, normalizer),
                                      data, m,
                                      reduction='none'),
                       dim=(0,2,3)).cpu().detach().numpy()
        loss_test[m].append(l)

for m in metric:
    loss_test[m] = np.stack(loss_test[m], axis=0)


In [8]:
loss_test_means = {m:loss_test[m].mean(axis=0) for m in metric}
loss_test_means

{'RMSE': array([1.261758  , 0.67746025, 0.6826467 , 0.35513178], dtype=float32),
 'MSE': array([3.9594295 , 0.96915835, 1.0503665 , 0.47758245], dtype=float32),
 'MAE': array([1.261758  , 0.67746025, 0.6826467 , 0.35513178], dtype=float32)}

In [9]:
loss_test_vars = {m:loss_test[m].var(axis=0) for m in metric}
loss_test_vars

{'RMSE': array([0.04755922, 0.00122973, 0.01319443, 0.00312377], dtype=float32),
 'MSE': array([3.3731494 , 0.03683224, 0.19296293, 0.0391564 ], dtype=float32),
 'MAE': array([0.04755922, 0.00122973, 0.01319443, 0.00312377], dtype=float32)}